### Избавляемся от лишних символов и оставляем только raw_text

In [2]:
cyrilic_letters = "аАбБвВгГдДеЕёЁжЖзЗиИйЙкКлЛмМнНоОпПрРсСтТуУфФхХцЦчЧшШщЩъЪыЫьЬэЭюЮяЯ"

In [3]:
right_symbols = string.ascii_letters + cyrilic_letters + ' '
data = []
for path in tqdm(files):
    entry = {}
    entry['id'] = path.split('/')[-1].split('.')[0]
    with open(path,'r') as f:
        next(f)
        next(f)
        text = f.read().replace("\n", ' ')
        text = ''.join([char for char in text if char in right_symbols])
        text = ' '.join(text.split())
        entry['raw_text'] = ''.join(text.split('\n'))
    data.append(entry)
habr_texts = pd.DataFrame(data)
print(habr_texts.columns)


Index(['id', 'raw_text'], dtype='object')


### Делаем токенизацию

In [4]:
tokenized_text = []
tokenizer = SpacyRulesRussianTokenizer()
for text in tqdm(habr_texts['raw_text'].values):
    text = tokenizer.transform_element(text)
    tokenized_text.append(text)
habr_texts['tokenized'] = tokenized_text

### Делаем лемматизацию

In [5]:
lemmatized_text = []
lemmatizer = Pymorphy2Lemmatizer()
for text in tqdm(habr_texts['tokenized'].values):
    lemmatized = lemmatizer.transform_string(text)
    lemmatized_text.append(lemmatized)
habr_texts['lemmatized'] = lemmatized_text

### Переводим лемматизированный текст к Vowpal Wabbit

In [6]:
vw_text = []
for index, data in tqdm(habr_texts.iterrows()):
    vw_string = ''
    doc_id = data.id
    lemmatized = '@word ' + ' '.join(data.lemmatized)
    vw_string = ' |'.join([doc_id, lemmatized])
    vw_text.append(vw_string)
habr_texts['vw_text'] = vw_text

### Сохраняем в формате csv

In [7]:
#DATA_PATH_SUB = 'DataPreparation.csv'
DATA_PATH_SUB = 'DataPreparation.csv'
habr_texts[['id','raw_text', 'vw_text']].to_csv(DATA_PATH_SUB)

### Получаем словарь с помощью get_dictionary для последующей "очистки" текста от слишком частотных и слишком редких слов

In [8]:
dataset = Dataset(DATA_PATH_SUB)
dictionary = dataset.get_dictionary()

/home/andreizoltan/anaconda3/lib/python3.6/site-packages/topicnet/cooking_machine/dataset.py:343: UserWarning: Attempted to use batches for different dataset.Overwriting batches in DataPreparation_test_batches
  warnings.warn(W_DIFF_BATCHES_1 + W_DIFF_BATCHES_2.format(batch_vectorizer_path))


In [9]:
dictionary.save_text("dict.txt")
! head dict.txt

name: dd50a3e6-acd7-4810-9a92-ca00f6939824 num_items: 10706
token, class_id, token_value, token_tf, token_df
порночат, @word, 4.954623022968008e-07, 1.0, 1.0
pacenza, @word, 4.954623022968008e-07, 1.0, 1.0
дрессировать, @word, 4.954623022968008e-07, 1.0, 1.0
клоновслон, @word, 4.954623022968008e-07, 1.0, 1.0
закладокфильмовальбом, @word, 4.954623022968008e-07, 1.0, 1.0
iac, @word, 4.954623022968008e-07, 1.0, 1.0
музыкуслов, @word, 4.954623022968008e-07, 1.0, 1.0
ногусвеловщик, @word, 4.954623022968008e-07, 1.0, 1.0


In [10]:
df = pd.read_csv("dict.txt", skiprows=[0])
df.shape

(83880, 5)

In [11]:
df[df[" class_id"] == ' @word'].sort_values(by=[" token_df"]).tail()

,token,class_id,token_value,token_tf,token_df
43377,что,@word,0.011774,23764.0,6773.0
41155,с,@word,0.011891,23999.0,7106.0
63504,на,@word,0.017903,36133.0,8301.0
63142,и,@word,0.028514,57551.0,8846.0
38627,в,@word,0.033660,67937.0,9184.0


### Выбираем слова от которых мы хотим избавиться и обновляем Vowpal Wabbit

In [12]:
bad_words = list()
for i, data in tqdm(df[df[" token_tf"] > 234].iterrows()):
    bad_words.append(df["token"][i])
for i, data in tqdm(df[df[" token_tf"] < 3].iterrows()):
    bad_words.append(df["token"][i])
print(len(bad_words))


57042


In [13]:
vw_text = []
for index, data in tqdm(habr_texts.iterrows(), total=habr_texts.shape[0]):
    vw_string = ''
    doc_id = data.id
    good_words = [word for word in data.lemmatized if word not in bad_words]
    lemmatized = '@word ' + ' '.join(good_words)
    vw_string = ' |'.join([doc_id, lemmatized])
    vw_text.append(vw_string)
habr_texts['vw_text'] = vw_text

### Записываем в файл

In [14]:
DATA_PATH = 'ReadyData.csv'
habr_texts[['id','raw_text', 'vw_text']].to_csv(DATA_PATH)